In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
import sys, os, pathlib
os.environ['PKG_CONFIG_PATH'] = '/ocean/projects/asc170022p/mtragoza/mambaforge/envs/lung-project/lib/pkgconfig'

import numpy as np
import pandas as pd
import nibabel as nib
import torch

sys.path.append('../..')
import project

torch.cuda.is_available()

/ocean/projects/asc170022p/mtragoza/mambaforge/envs/warp/lib/python3.11/site-packages/ufl/__init__.py:250: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


True

In [3]:
!pwd

/ocean/projects/asc170022p/mtragoza/lung-project/notebooks/copdgene


In [7]:
!ls ../../data/ShapeNetSem

categories.synset.csv  metadata.csv    models-binvox	    taxonomy.txt
densities.csv	       models-COLLADA  models-binvox-solid
materials.csv	       models-OBJ      models-textures


In [11]:
!ls models-OBJ

ls: cannot access 'models-OBJ': No such file or directory
